<a href="https://colab.research.google.com/github/sakurasakura1996/Pytorch-start-learning/blob/master/Dive_into_DL_pytorch_4_4_%E8%87%AA%E5%AE%9A%E4%B9%89%E5%B1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 深度学习的一个魅力在于神经网络中各式各样的层，例如全连接层和后面章节中将要介绍的卷积层、池化层、与循环层
# 虽然Pytorch提供了大量常用的层，但有时候我们依然希望自定义层。本节将介绍如何使用Module来自定义层，从而可以被重复调用
# ￥ 4.4.1 不含模型参数的自定义层
# 下面自定义的层继承Module类 定义了一个将输入减掉均值后输出的层，并将层的计算定义在了forward函数里。这个层里不包含模型参数
import torch
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self, **kwargs):
        super(CenteredLayer,self).__init__(**kwargs)
    def forward(self, x):
        return x - x.mean()

In [0]:
layer = CenteredLayer()
layer(torch.tensor([1, 2, 3, 4, 5], dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

In [0]:
# 还可以同他来构造更复杂的模型
net = nn.Sequential(nn.Linear(8,128),layer)
print(net)
y = net(torch.rand(4,8))
print(y.size())
print(y.mean())
print(y.mean().item())

Sequential(
  (0): Linear(in_features=8, out_features=128, bias=True)
  (1): CenteredLayer()
)
torch.Size([4, 128])
tensor(-4.6566e-09, grad_fn=<MeanBackward0>)
-4.6566128730773926e-09


In [0]:
# 4.4.2 含模型参数的自定义层
# 不含模型参数的自定义层可以在forward中定义，含模型参数的自定义层可以通过训练学出，这叫要在
# 初始化时就定义了。 4.2节中介绍了Parameter类其实是Tensor的子类。如果一个Tensor是Parameter，那么它会被自动添加到参数
# 列表中去。所以在自定义含模型参数的层时，我们应该将参数定义成Parameter，除了像4.2.1直接定义成parameter类外，还可以使用
# ParameterList和ParameterDict分别定义参数的列表和字典
class MyDense(nn.Module):
    def __init__(self):
        super(MyDense, self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.rand(4,4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4,1)))
    
    def forward(self, x):
        for i in range(len(self.params)):
            x = torch.mm(x, self.params[i])
        return x

net = MyDense()
print(net)

MyDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


In [0]:
# 而 ParameterDict 接受一个 Parameter实例的字典作为输入然后得到一个参数字典，然后可以按照字典的规则使用了。例如使用 update()新增参数，使用keys()返回所有
# 键值，使用items()返回所有键值对等等。
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense, self).__init__()
        self.params = nn.ParameterDict({
            'linear1':nn.Parameter(torch.rand(4,4)),
            'linear2':nn.Parameter(torch.rand(4,4)),
        })
        self.params.update({
            'linear3':nn.Parameter(torch.rand(4,1))
        })
        
    def forward(self,x):
#         for key in self.params.keys():
#             x = torch.mm(x, self.params[key])  这两种都可以啦
        for key, value in self.params.items():
            x = torch.mm(x, self.params[key])
        return x
    
#     def forward(self, x, choice='linear1'):
#         # 这里是原书中的前向传播代码，这里更灵活一些，因为整个网络中的结构，我们不一定非要经过整个网络，而可以选择性的通过网络部分
#         return torch.mm(x,self.params[choice])

net = MyDictDense()
print(net)
x = torch.rand(1,4)
print(net(x))
# print(net(x, 'linear3'))

MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)
tensor([[2.4291]], grad_fn=<MmBackward>)


In [0]:
# 我们也可以使用自定义层构造模型，它和pytorch 的其他层在使用上很类似
net = nn.Sequential(
    MyDictDense(),
    MyDense(),
)
print(net)


Sequential(
  (0): MyDictDense(
    (params): ParameterDict(
        (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
        (linear2): Parameter containing: [torch.FloatTensor of size 4x4]
        (linear3): Parameter containing: [torch.FloatTensor of size 4x1]
    )
  )
  (1): MyDense(
    (params): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 4x4]
        (1): Parameter containing: [torch.FloatTensor of size 4x4]
        (2): Parameter containing: [torch.FloatTensor of size 4x4]
        (3): Parameter containing: [torch.FloatTensor of size 4x1]
    )
  )
)
